In [1]:
import ray

In [2]:
ray.init(num_cpus=4)

2023-07-03 17:57:11,105	INFO worker.py:1636 -- Started a local Ray instance.


Python version:,3.9.16
Ray version:,2.5.0


In [3]:
# ray.shutdown()

In [1]:
import pandas as pd
%cd ..

import logging
from bds.meel import approx_mc2, approx_mc2_core
from bds.bb import get_ground_truth_count
from bds.rule import Rule
from bds.utils import bin_random, randints
from logzero import logger

import numpy as np
from contexttimer import Timer

logger.setLevel(logging.WARN)

num_pts = 1000

ub = 0.8
lmbd = .1

delta = .8
eps = .8

show_progres = True
rand_seed = 1234

/u/50/ciaperm1/unix/sampling-rashomon-decision-set-code


In [8]:
n_reps = 1
num_rules_list = list(reversed([50]))
# num_rules_list = list(reversed([200]))


np.random.seed(rand_seed)
res_rows = []
for _ in range(n_reps):
    for num_rules in num_rules_list:
        rule_random_seeds = randints(num_rules)
        rules = [
            Rule.random(i+1, num_pts, random_seed=rule_random_seeds[i])
            for i in range(num_rules)
        ]
        y = bin_random(num_pts)

        with Timer() as cbb_timer:
            test_cnt = approx_mc2(
                rules, y, lmbd=lmbd, ub=ub,
                delta=delta, eps=eps, rand_seed=rand_seed, show_progress=show_progres,
                parallel=False, log_level=logging.WARN
            )
        test_elapsed = cbb_timer.elapsed
        
        with Timer() as bb_timer:
            ref_count = get_ground_truth_count(rules, y, lmbd, ub)
        ref_elapsed = bb_timer.elapsed        

        res_rows.append((num_rules, test_elapsed, ref_elapsed, test_cnt, ref_count))

  0%|                                                    | 0/32 [00:00<?, ?it/s]

rand_seed: 822569775


  3%|█▍                                          | 1/32 [00:00<00:07,  4.25it/s]

time_cost_info: 
|1|0.004279395041521639|
|2|0.0071799830184318125|
|3|0.010030820034444332|
|4|0.01607514900388196|
|8|0.06263573403703049|
|6|0.05963210598565638|
|7|0.05985242401948199|
rand_seed: 2137449171


  9%|████▏                                       | 3/32 [00:00<00:06,  4.67it/s]

...

time_cost_info: 
|1|0.0027191860135644674|
|2|0.0034597910125739872|
|3|0.005358503025490791|
|4|0.009539944992866367|
|8|0.055994256981648505|
|6|0.07086088199866936|
|5|0.017734553024638444|
rand_seed: 421988862
time_cost_info: 
|1|0.0028436900465749204|
|2|0.003818147990386933|
|3|0.007349883962888271|
|4|0.01361437002196908|
|8|0.061236661975272|
|6|0.037639342015609145|
|7|0.06048735196236521|
rand_seed: 262835631


 97%|█████████████████████████████████████████▋ | 31/32 [00:07<00:00,  4.45it/s]

time_cost_info: 
|1|0.003229298978112638|
|2|0.004395221010781825|
|3|0.006664216984063387|
|4|0.011419354006648064|
|8|0.07944501197198406|
|6|0.11377956497017294|
|5|0.04394970997236669|
rand_seed: 240490034


100%|███████████████████████████████████████████| 32/32 [00:07<00:00,  4.34it/s]

time_cost_info: 
|1|0.005338191986083984|
|2|0.00733321305597201|
|3|0.010892474034335464|
|4|0.01579810999101028|
|8|0.0663247830234468|
|6|0.06332964496687055|
|7|0.0732772090123035|


In [6]:
ref_count

4902

In [7]:
test_cnt

4672

In [8]:
len(rules)

50

In [2]:
from bds.bbSampleTreeApproxCounting import BranchAndBoundNaive
import time 
import random 

In [6]:
for l in [1]: 
    for k in [6500]: 
        bb = BranchAndBoundNaive(rules, ub, y, lmbd, l=l, k=k) 
        Z = bb.runST()
        print("count " + str(Z))

NameError: name 'rules' is not defined

In [ ]:
all_counts = [] 
for _ in range(100): 
    for l in [1]: 
        for k in [1500]: 
            bb = BranchAndBoundNaive(rules, ub, y, lmbd, l=l, k=k) 
            Z = bb.runST()
            all_counts.append(Z) 
    

In [ ]:
np.mean(all_counts), np.median(all_counts)

In [ ]:
df = pd.DataFrame(
    res_rows,
    columns=['num_rules', 'running_time_approx_mc2', 'running_time_bb', 'estimate_count', 'true_count']
)
df['runtime-factor'] = df['running_time_approx_mc2'] / df['running_time_bb']
df['estimation-rel-diff'] = (df['estimate_count'] - df['true_count']) / df['true_count']

df



In [ ]:
print(df.groupby('num_rules')['running_time_approx_mc2', 'running_time_bb', 'runtime-factor'].mean().to_markdown())

In [ ]:
y

In [3]:
num_rules = 2
rule_random_seeds = randints(num_rules)
rules = [
    Rule.random(i+1, num_pts, random_seed=rule_random_seeds[i])
    for i in range(num_rules)
]
y = bin_random(num_pts)

In [4]:
y = np.array([0, 0, 1, 0, 1], dtype=bool)
from bds.rule import Rule
from bds.utils import mpz_set_bits
from gmpy2 import mpz, mpfr


rules = [
        Rule(
            id=1,
            name="rule-1",
            cardinality=1,
            # truthtable=np.array([0, 1, 0, 1, 0], dtype=bool),
            truthtable=mpz_set_bits(mpz(), [1, 3])
        ),
        Rule(
            id=2,
            name="rule-2",
            cardinality=1,
            # truthtable=np.array([0, 0, 1, 0, 1], dtype=bool),
            truthtable=mpz_set_bits(mpz(), [2, 4])
        ),
        Rule(
            id=3,
            name="rule-3",
            cardinality=1,
            # truthtable=np.array([1, 0, 1, 0, 1], dtype=bool),
            truthtable=mpz_set_bits(mpz(), [0, 2, 4])
        ),
    ]



In [5]:
bb = BranchAndBoundNaive(rules, 10 , y, lmbd, l=1, k=100000) 
Z = bb.runST()

calling!
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
calling!
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
calling!
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
calling!
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
4

In [30]:
2**3 - 1

7

In [28]:
len([{0, 1}, {0}, {0,1,2}, {0,2}, {0,1,3}, {0,3}, {0,1,2,3}, {0,2,3}])

8

In [31]:
Z

8.0